# Problem Set 4

## Problem 1: Galaxy Photometric Redshifts (6 points)

Let us return to an example from cosmology. Many of the galaxies we observe with telescopes are millions of light years away. Because the Universe has expanded since the light of a galaxy was emitted, its light is shifted towards longer wavelengths, i.e., redder parts of the electromagnetic spectrum. This effect, which is called redshift, can be measured precisely with spectroscopy. However, spectroscopy is time-consuming and challenging for a large number of galaxies. Thus, a common problem in astronomy is trying to estimate the redshift of a galaxy by looking only at its flux or intensity in different parts, so-called bands, of the electromagnetic spectrum. Redshifts estimated this way are called photometric redshifts.

The file `Problem_Set_4_Redshifts.csv` contains the following data.
* `<BAND>_FLUX`: Fluxes in different bands. This is what will be used for training. Note that the data has been cleaned, and galaxies with missing fluxes have been removed.
* `Z_PHOT`: This is the redshift estimated from a state-of-the-art photometric redshift code, [LePHARE](https://lephare.readthedocs.io/en/latest/). You should **not** use this during training or validation. It's simply another estimate to compare against.
* `Z_SPEC`: These are spectroscopic redshifts $z_\mathrm{spec}$ measured from the Dark Energy Spectroscopic Instrument (DESI).

(For those interested, fluxes and photometric redshifts come from the [COSMOS2020 catalog](https://cosmos2020.calet.org/). The spectroscopic redshifts are available [here](https://data.desi.lbl.gov/public/papers/c3/cosmos-xmmlss/). The script `make_problem_set_4.py` "reduces" the publicly available data sets to `Problem_Set_4_Redshifts.csv`.)

(a) Make a histogram of the spectroscopic redshifts, $z_\mathrm{spec}$. Additionally, compute the $R^2$ score of a simple linear model as a baseline to compare against.

(b) Use a random forest to predict the spectroscopic redshift from the measured fluxes. Perform a $k$-fold cross-validation with $k=10$ to optimize hyperparameters based on the $R^2$ score. Include (+/- 1 standard deviation) error bars derived from the scatter between different folds. To keep the compute time reasonable, limit the number of trees in the random forest to $30$. How does the score of the random forest compare against the linear model? I'm not grading you on having found the absolute best hyperparameters as long as the cross-validation search is reasonable, that is, testing a few values for $3$ or $4$ hyperparameters within reasonable ranges. (Tip: You probably want to use `n_jobs=-1` in `sklearn.ensemble.RandomForestRegressor` to make the random forest use as many CPU cores as possible.)

(c) Compute the cross-validated $R^2$ training curve for the best-performing hyperparameters with with the number of training samples $n$ going from $1000$ to $10000$. As for (b), please include error bars. Does this curve indicate significant overfitting? Would our random forest redshifts benefit meaningfully from more training data?

(d) In astronomy, we are often interested in the catastrophic outlier rate $\eta$ of redshifts. This rate is defined as the fraction of photometric redshifts $\hat z$ for which $|\hat z - z_\mathrm{spec}| > 0.15 (1 + z_\mathrm{spec})$. Thus, another "score" we are interested in is $1 - \eta$, the fraction of redshifts that are not catastrophic outliers. Plot the learning curve for the cross-validated $1 - \eta$ score and indicate the catastrophic outlier rate of the LePHARE photometric redshifts in the same plot. (Hint: `sklearn.model_selection` can take a custom score function via the `scoring` keyword argument. More information can be found [here](https://scikit-learn.org/stable/modules/model_evaluation.html#custom-scorer-objects-from-scratch).)

(e) Visualize the feature importance in the random forest. What does this indicate about the importance of different photometric bands?

In [49]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv("Problem_Set_4_Redshifts.csv")
y = df["Z_SPEC"]
X = df.iloc[:, :35] # all fluxes
y_est = df["Z_PHOT"]
flux_names = X.columns

def a():
    plt.yscale("log")
    plt.hist(df["Z_SPEC"], bins = 50)
    plt.title("Spectroscopic Redshift Distribution")
    plt.xlabel("Z Spec")
    plt.ylabel("Count")
    plt.show()

    reg = LinearRegression().fit(X, y)
    y_pred = reg.predict(X)
    linear_r2 = metrics.r2_score(y_est, y_pred)
    print("Linear Regression R2 Score:", linear_r2)

#### unsure about what param_grid should be
def b():
    k_num = 10
    kfold = model_selection.KFold(n_splits=k_num, shuffle=True)
    random_forest = RandomForestRegressor(n_estimators=30, n_jobs=-1)
    model = model_selection.GridSearchCV(random_forest, list(flux_names), cv = kfold, scoring = "r2").fit(X, y)
    forest_r2 = model.cv_results["mean_test_score"]
    print("Random Forest R2 Score:", forest_r2)

b()

TypeError: Parameter grid is not a dict ('CFHT_u_FLUX')